In [9]:
import os
import speech_recognition as sr
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.utilities import OpenWeatherMapAPIWrapper,WikipediaAPIWrapper, DuckDuckGoSearchAPIWrapper
from langchain.agents import Tool
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain.memory import ConversationBufferMemory
import pyttsx3 
from langchain_core.runnables.history import RunnableWithMessageHistory #Memory function
from langsmith import utils

In [10]:
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
recognizer = sr.Recognizer()

In [11]:
OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"]="https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"]="Travel Assistant"



In [12]:
utils.tracing_is_enabled()

True

In [13]:
weather_api = OpenWeatherMapAPIWrapper()  #https://home.openweathermap.org/api_keys

wiki_api = WikipediaAPIWrapper()


In [14]:
embed = OpenAIEmbeddings(model='text-embedding-ada-002') 
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
    

In [15]:
def wiki_search(query):
    data = wiki_api.load(query)
    split_docs = splitter.split_documents(data)
    vectorstore = FAISS.from_documents(split_docs, embed)

    return vectorstore.as_retriever()

In [16]:
def retriever_tool(query):
    retriever = wiki_search(query)
    results = retriever.get_relevant_documents(query)
    return "\n".join([doc.page_content for doc in results])


In [17]:
from langchain.llms import OpenAI
llm = OpenAI(temperature=0, model_name="gpt-3.5-turbo-instruct")  # or "gpt-4"


C:\Users\riina\AppData\Local\Temp\ipykernel_28508\2785385720.py:2: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm = OpenAI(temperature=0, model_name="gpt-3.5-turbo-instruct")  # or "gpt-4"


In [18]:
from langchain.prompts import PromptTemplate

# Define the custom prompt template for SmartTourGuideAgent
tour_guide_prompt = PromptTemplate(
    input_variables=["query"],
    template="""
    You are a SmartTourGuideAgent. Your task is to help users with their travel-related questions.
    If the user asks about a city or country, provide them with relevant details about it.
    If the user asks for a trip plan, create a basic itinerary for them.

    Answer questions with relevant emojis to make the answers more engaging and fun. For example:
    - For cities, mention landmarks, weather, or activities with emojis.
    - For countries, mention famous attractions or cultural highlights with emojis.
    - For trip plans, create a fun schedule with emojis representing the activities.

    Query: {query}
    Answer:
    """
)

In [19]:
tools = [
    Tool(
    name="TravelInfoRetriever",
    func=retriever_tool,
    description=(
    "You are a SmartTourGuideAgent."
    "If the user asks about a city or country, provide them with details about it."
    "If the user asks for a trip plan, create a basic itinerary for them."
    "- For cities, mention landmarks, weather, or activities with emojis."
    "- For countries, mention famous attractions or cultural highlights with emojis."
    "- For trip plans, create a fun schedule with emojis representing the activities."
    )
    ),

  Tool(
        name='Weather',
        func= weather_api.run,
        description=(
        "Use this tool to find **current or forecasted weather information** about a country, city, or travel destination. "
        "Ideal for questions like: 'What's the weather in Rome?', 'Is it rainy in Tokyo?', or 'How cold is it in Iceland in December?'. "
        "Only use this tool when the user is asking specifically about **weather** conditions. "
        "Do not use it for general travel info or sightseeing.")
  )
]

In [20]:
# 3. Audio Processing Functions
def listen_to_user(timeout=10):
    """Convert speech to text with error handling"""
    try:
        with sr.Microphone() as source:
            print("\n🎤 Listening... (Speak now)")
            audio = recognizer.listen(source, timeout=timeout)
        return recognizer.recognize_google(audio)
    except sr.WaitTimeoutError:
        print("⌛ No speech detected, please try again!")
        return ""
    except Exception as e:
        print(f"🔇 : {str(e)}. Try speaking again.")
        return ""

def text_to_speech(text):
    """Convert text to speech"""
    engine = pyttsx3.init()
    engine.setProperty('rate', 150)  # Speed of speech
    engine.setProperty('volume', 1)  # Volume level (0.0 to 1.0)
    engine.say(text)
    engine.runAndWait()

def get_input_method():
    """Get user's preferred input method"""
    while True:
        method = input("\nChoose input method [text/audio]: ").lower()
        if method in ["text", "t"]:
            return input("✍️ Your travel question: ")
        elif method in ["audio", "a"]:
            return listen_to_user()
        print("⚠️ Please enter 'text' or 'audio'")


In [21]:
from langchain.agents import initialize_agent
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(memory_key="chat_history")

conversational_agent = initialize_agent(
    agent="conversational-react-description",
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=6,
    agent_kwargs={
        "prompt": tour_guide_prompt
    },
     memory=memory  # Pass the memory object here
)

C:\Users\riina\AppData\Local\Temp\ipykernel_28508\2498294467.py:6: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  conversational_agent = initialize_agent(


In [22]:
conversational_agent.run(get_input_method())

C:\Users\riina\AppData\Local\Temp\ipykernel_28508\168107454.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  conversational_agent.run(get_input_method())




> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: TravelInfoRetriever
Action Input: make a plan

C:\Users\riina\AppData\Local\Temp\ipykernel_28508\741416126.py:3: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  results = retriever.get_relevant_documents(query)



Observation: A plan is typically any diagram or list of steps with details of timing and resources, used to achieve an objective to do something. It is commonly understood as a temporal set of intended actions through which one expects to achieve a goal.
For spatial or planar topologic or topographic sets see map.
Plans can be formal or informal:
Structured and formal plans, used by multiple people, are more likely to occur in projects, diplomacy, careers, economic development, military campaigns, combat, sports, games, or in the conduct of other business. In most cases, the absence of a well-laid plan can have adverse effects: for example, a non-robust project plan can cost the organization time and money.
Informal or ad hoc plans are created by individuals in all of their pursuits.
It is common for less formal plans to be created as abstract ideas, and remain in that form as they are maintained and put to use. More formal plans as used for business and military purposes, while initi

'Sure, I can help you make a plan. What do you need a plan for?'